In [ ]:
import pandas as pd
from sqlalchemy import create_engine, Column, String, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from fastapi import FastAPI, HTTPException

# Load Excel file
file_path = r"C:\Users\austin\Desktop\PythonLearn\Remi\Interns_2025_SWIFT_CODES.xlsx"
df = pd.read_excel(file_path)

# Normalize column names
df.columns = df.columns.str.strip().str.replace(" ", "_").str.upper()
df.head()


,COUNTRY_ISO2_CODE,SWIFT_CODE,CODE_TYPE,NAME,ADDRESS,TOWN_NAME,COUNTRY_NAME,TIME_ZONE
0,AL,AAISALTRXXX,BIC11,UNITED BANK OF ALBANIA SH.A,"HYRJA 3 RR. DRITAN HOXHA ND. 11 TIRANA, TIRANA...",TIRANA,ALBANIA,Europe/Tirane
1,BG,ABIEBGS1XXX,BIC11,ABV INVESTMENTS LTD,"TSAR ASEN 20 VARNA, VARNA, 9002",VARNA,BULGARIA,Europe/Sofia
2,BG,ADCRBGS1XXX,BIC11,ADAMANT CAPITAL PARTNERS AD,"JAMES BOURCHIER BLVD 76A HILL TOWER SOFIA, SOF...",SOFIA,BULGARIA,Europe/Sofia
3,UY,AFAAUYM1XXX,BIC11,AFINIDAD A.F.A.P.S.A.,"PLAZA INDEPENDENCIA 743 MONTEVIDEO, MONTEVIDE...",MONTEVIDEO,URUGUAY,America/Montevideo
4,MC,AGRIMCM1XXX,BIC11,CREDIT AGRICOLE MONACO (CRCA PROVENCE COTE D'A...,"23 BOULEVARD PRINCESSE CHARLOTTE MONACO, MONA...",MONACO,MONACO,Europe/Monaco


In [10]:

Base = declarative_base()

class SwiftCode(Base):
    __tablename__ = "swift_codes"

    swift_code = Column(String, primary_key=True)
    bank_name = Column(String)
    country_iso2 = Column(String)
    country_name = Column(String)
    address = Column(String)
    is_headquarter = Column(Boolean)

# Initialize database
DATABASE_URL = "sqlite:///./swift_codes.db"
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
Base.metadata.create_all(bind=engine)

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)


C:\Users\austin\AppData\Local\Temp\ipykernel_20392\1542872589.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [11]:

# Identify headquarters and branches
df["IS_HEADQUARTER"] = df["SWIFT_CODE"].str.endswith("XXX")

# Store in database
session = SessionLocal()
for _, row in df.iterrows():
    swift_entry = SwiftCode(
        swift_code=row["SWIFT_CODE"],
        bank_name=row["NAME"],
        country_iso2=row["COUNTRY_ISO2_CODE"],
        country_name=row["COUNTRY_NAME"],
        address=row["ADDRESS"],
        is_headquarter=row["IS_HEADQUARTER"],
    )
    session.merge(swift_entry)

session.commit()
session.close()


In [12]:

app = FastAPI()

@app.get("/v1/swift-codes/{swift_code}")
def get_swift_code(swift_code: str):
    session = SessionLocal()
    swift_entry = session.query(SwiftCode).filter_by(swift_code=swift_code).first()
    session.close()
    
    if not swift_entry:
        raise HTTPException(status_code=404, detail="SWIFT code not found")

    response = {
        "swiftCode": swift_entry.swift_code,
        "bankName": swift_entry.bank_name,
        "countryISO2": swift_entry.country_iso2,
        "countryName": swift_entry.country_name,
        "address": swift_entry.address,
        "isHeadquarter": swift_entry.is_headquarter,
    }
    
    return response

# Run the FastAPI server
!uvicorn main:app --reload --port 8080


INFO:     Will watch for changes in these directories: ['c:\\Users\\austin\\Desktop\\PythonLearn\\Remi']
ERROR:    [WinError 10013] An attempt was made to access a socket in a way forbidden by its access permissions
